In [ ]:
#-*- coding: utf-8 -*-
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
# 将列表中的数据切片读入矩阵
def Read(lines,m,n):
    A = np.zeros((m, n))
    A_row = 0  # 表示矩阵的行，从0行开始
    for line in lines:  # 把lines中的数据逐行读取出来
        list = line.strip('\n').split('\t')  # 处理逐行数据：strip表示把头尾的'\n'去掉，split表示以空格来分割行数据，然后把处理后的行数据返回到list列表中
        A[A_row:] = list[0:5]  # 把处理后的数据放到方阵A中。list[0:4]表示列表的0,1,2,3列数据放到矩阵A中的A_row行
        A_row += 1  # 然后方阵A的下一行接着读
    return A


In [ ]:

'''
1.读取训练集和测试集
注:在此处统计行数是为了兼容不同的样本集,
因为理论上说我们事先不会知晓有多少组数据
'''
f1 = open('F:\\Code\\Mode Regonization\\Iris\\Fisher\\train.txt') # 打开训练集
f2 = open('F:\\Code\\Mode Regonization\\Iris\\Fisher\\test.txt') # 打开测试集
lines1 = f1.readlines() # 把全部数据文件读到一个列表lines中
lines2 = f2.readlines()
Line1 = len(lines1) # 读取训练集行数
Line2 = len(lines2) # 读取训练集列数
A = Read(lines1,Line1,5)
B = Read(lines2,Line2,5)

In [ ]:
'''
2.将三类样本拆分
'''
# 提取三类训练集
A1,A2,A3 = A[0:25],A[25:50],A[50:75]
# 提取三类测试集
B1,B2,B3 = B[0:25],B[25:50],B[50:75]
# A2 = A3

In [ ]:
'''
2.去除无用的数据
例如目前需要绘制第一类和第二类的分类面，选取前三个特征进行分类
就需要去除第一行（标签）和第五行的数据（第四个特征）
'''
A1 = np.delete(A1,[0,4],axis=1)
A2 = np.delete(A2,[0,4],axis=1)
a1,a2 = A1,A2 # 寄存后续绘制散点图使用

In [ ]:
'''
3.样本增广化、规范化处理
'''
#样本增广化
One = np.ones(25) # 1*25的行向量值均为1
A1 = np.insert(A1,0,values=One,axis=1) # 在第0列插入One向量（变成了列向量）
A2 = np.insert(A2,0,values=One,axis=1)
#样本规范化
A = np.concatenate((A1,-1*A2),axis=0) # A1与A2矩阵拼接在一起
# print(A)


In [ ]:
'''
4.采用单步处理的梯度下降算法进行迭代运算
计算权向量w和迭代次数k
w为4*1的向量,初始化时设定为[1,1,1,1]^T
设定学习率p
'''
w = np.ones((4,1))
k = 1 # 迭代次数
p = 2 # 学习率
count = 0 # 记录数据集里连续分类正确的个数，count=50代表全部分类成功
i = 0 # 目前在用第i个数据检验正确性
# '''开始迭代！！'''
while 1:
    y = np.dot(A[i,:],w)
    if y <= 0:
        J = -y
        w = w + p * np.mat(A[i,:]).transpose() 
        count = 0 # 分类错误，count重置
        print("第%d次迭代中,求解惩罚值为%d"%(k,J))
        print("更新权向量值为:",w.transpose())
    else:
        J = 0
        count = count + 1
        print("第%d次迭代中,该权向量对目前所检验的样本分类正确"%k)
        print("权向量值依旧为:",w.transpose())
    '''判断是否迭代完毕'''
    if count==50: # 连续50个样本分类正确
        print("迭代完毕！")
        break
    '''下一轮的迭代'''
    k = k + 1
    i = (i + 1) % 50
print("经过%d次迭代后找到最终权向量求解值为:"%k,w.transpose())




In [ ]:
'''
5.对训练数据绘制散点图
绘制分类面
三个特征的权向量格式为w=[w0,w1,w2,w3]
则可得分类面方程为 : w1*x + w2*y + w3*z + w0 = 0
推导可得 -w3*z = w1*x + w2*y + w0
z = -(w1*x + w2*y + w0)/w3
'''
fig = plt.figure()
ax = plt.axes(projection='3d')
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
point1 = ax.scatter3D(a1[:,0].tolist(),a1[:,1].tolist(),a1[:,2].tolist(),c='red',marker = '+')  # type: ignore
point2 = ax.scatter3D(a2[:,0].tolist(),a2[:,1].tolist(),a2[:,2].tolist(),c='yellow',marker = '+')  # type: ignore
x = np.linspace(-1,6,10)
y = np.linspace(-1,6,10)
X,Y = np.meshgrid(x,y)
w = w.A # 矩阵单个元素也属于一个独立矩阵，因此将matrix矩阵类型转为ndarray
Z = -1*(w[1]*X+w[2]*Y+w[0])/w[3]
ax.plot_surface(X,Y,Z,color='aqua')  # type: ignore
ax.set(xlabel="X", ylabel="Y", zlabel="Z")
# m = w.A
# m = np.mat(m)
# print(m)

In [ ]:
# 计算准确率
def count_accuracy(true,false,id,Id):
    if id==Id:
        true+=1
    else:
        false+=1
    return true,false
'''
6.使用导入的测试集进行检验
'''
del_line = 4
ID = [1,2]
flag = 1;
true,false =0,0
# 去除无用的数据
B1 = np.delete(B1,del_line,axis=1)
B2 = np.delete(B2,del_line,axis=1)
# 样本增广化处理
One = np.ones(25) # 1*25的行向量值均为1
B1 = np.insert(B1,1,values=One,axis=1)
B2 = np.insert(B2,1,values=One,axis=1)
B = np.concatenate((B1,B2),axis=0) # B1与B2矩阵拼接在一起
res1,res2 = [],[]
for i in range(0,50):
    B_row = B[i] # 取出第i行
    id = B_row[0] # 取出测试集实际标号
    B_row = np.delete(B_row,0) # 矩阵中删除类别号
    y = np.dot(B_row,w)
    if y > 0:
        res1.append(B_row)
        Id = ID[0]
    else:
        res2.append(B_row)
        Id = ID[1]
    true,false = count_accuracy(true,false,id,Id)
if flag==1:
    str='感知器批处理方法'
elif flag==2:
    str='感知器单步处理方法'
else:
    str='感知器最小平方误差判别方法'
print("基于%s对第%d类和第%d类进行分类："%(str,ID[0],ID[1]))
print("正确个数：",true)
print("错误个数：",false)
print("准确率：",true/(true+false),'\n')

In [ ]:
'''
对样本的训练函数
A--某一类的训练样本
w--存储着三类目前的权向量，为列表[w1,w2,w3]
right_id,该样本正确的id值
count--数据集里连续分类正确的个数
k--迭代次数 p--学习率
'''
def show_w(w):
    print("第一类:",w[0].transpose())
    print("第二类:",w[1].transpose())
    print("第三类:",w[2].transpose())
def Train(A,w,right_id,count,k,p):
    y = [np.dot(A1[i,:],w[0]),np.dot(A1[i,:],w[1]),np.dot(A1[i,:],w[2])]
    # 寻找最大值的下角标
    cur_id = y.index(max(y))
    # 如果此时最大值的下角标有cur_id + 1 = right_id，即判别类别与实际类别相同
    if cur_id + 1 == right_id:
        print("第%d次迭代中,该权向量对目前所检验的样本分类正确"%k)
        print("三类权向量值依旧为")
    # 如果此时y[cur_id]是最大的，需要减小w[cur_id]
    else:
        temp = p * np.mat(A[i,:]).transpose() 
        w[cur_id] = w[cur_id] - temp
        # 因为y[right_id-1]不是最大的，所以需要增加w[right_id-1]
        w[right_id - 1] = w[cur_id - 1] + temp
        count = 0 # 分类错误，count重置
        print("第%d次迭代中,该权向量对目前所检验的样本分类错误"%k)
        print("三类权向量值更新为")
    show_w(w)
